In [ ]:
R.home()

In [ ]:
baizer::pkglib(tidyverse, baizer, fs)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])
species <- config[['species']]

gex_ref <- str_glue('{getwd()}/src/{species}-gex-ref')
vdj_ref <- str_glue('{getwd()}/src/{species}-vdj-ref')

In [ ]:
# write feature_ref.csv
if (config[[sample]][['FB']]) {
    id2seq <- config[[sample]][['id2seq']]
    id2seq <- id2seq[!map_lgl(id2seq, is.null)]
    TBfb_ref <- map2_dfr(id2seq, names(id2seq), 
                         ~list2df(.x, colnames='sequence') %>% r2c('id') %>% mutate(type=.y)) %>%
        mutate(
            name = str_glue('{type}_{id}'),
            read = config[['FB_read']],
            pattern = config[['FB_pattern']],
            feature_type = config[['FB_type']]
              ) %>%
        select(id, name, read, pattern, sequence, feature_type)

    TBfb_ref %>% write_excel_csv(snakemake@params[['feature_ref']])
}

In [ ]:
indir <- path_abs(config[['indir']])
outdir <- path_abs(config[['outdir']])

In [ ]:
# write config.csv
filepath <- snakemake@output[['cellranger_config']]
dir.create(dirname(filepath), recursive = TRUE)
out <- file(filepath, 'w')

# reference
if (config[[sample]][['FB']] | config[[sample]][['mRNA']]) {
    writeLines(str_glue("[gene-expression]\nreference,{gex_ref}\nchemistry,SC5P-R2"), out)
    if (config[[sample]][['FB']]) {
        writeLines(str_glue("[feature]\nreference,{snakemake@params[['feature_ref']]}"), out)
    }
}

# vdj
if (config[[sample]][['VDJB']] | config[[sample]][['VDJT']]) {
    writeLines(str_glue("[vdj]\nreference,{vdj_ref}"), out)
}

# libraries
writeLines('[libraries]\nfastq_id,fastqs,lanes,feature_types,subsample_rate', out)
if (config[[sample]][['mRNA']]) {
    writeLines(str_glue("{sample}-mRNA,{indir}/{sample},,Gene Expression,,"), out)
}
if (config[[sample]][['FB']]) {
    writeLines(str_glue("{sample}-FB,{indir}/{sample},,{config[['FB_type']]},,"), out)
}
if (config[[sample]][['VDJB']]) {
    writeLines(str_glue("{sample}-VDJB,{indir}/{sample},,VDJ-B,,"), out)
}
if (config[[sample]][['VDJT']]) {
    writeLines(str_glue("{sample}-VDJT,{indir}/{sample},,VDJ-T,,"), out)
}

close(out)